### This is designed to merge the single papers together to assemble one bigger data-csv that we can at some point use for model training

# General Notes: 
- To execute the notebook, you frist need to produce the .csv from the underlying papers
- I try to convert and select for ENSG gene rows as we got  datasets from Flo with 60k entries which most probably will be me mayjority


In [51]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
from utils.helper import manhattanplot, violinplot_overall, scatter_plot, get_negative_values, METADATA_COLS, reorder_columns_by_metadata_and_gene_counts


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
### Wimmers 2023 (NOT INCLUDED)
#NOTE This is 1 of 2 Datasets from this paper. 
#NOTE Also this datasets should be clarified with Flo first. It´s PBMC but not clear from whom ? (maybe GSE239799 ??? but also different)
wimmers_2_df = pd.read_csv(f'/home/eliasschreiner/PROJEKT/2023_Wimmers_CoVid_multi-omics/2023_Wimmers_combi_df.csv')
print(wimmers_2_df.shape)
wimmers_2_df.head()
PLATFORM_GEO_ID_wimmers_23 = "GPL24676"
PLATFORM_DESCRIPTION_wimmers_23 = "Illumina NovaSeq 6000 (Homo sapiens)"
METHOD_wimmers_23 = "RNAseq"
#TODO in the data itself PBMC is mentioned but https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE239787 it is whole blood 
TYPE_wimmers_23 = "blood"
study_accession_wimmers_23 = "???"
desease_wimmers_23 = "SARS-CoV-2 / healthy"



(119, 60710)


In [57]:
### Wimmers 2023 
#NOTE This is 2 of 2 Datasets from this paper. 
wimmers_GEO239787_df = pd.read_csv(f'2023_Wimmers_CoVid_multi-omics/2023_Wimmers_GSE239787_GEO.csv', index_col=0)
print(wimmers_GEO239787_df.shape)
wimmers_GEO239787_df = wimmers_GEO239787_df.rename(columns={
    'ID':'subject_accession', 
    'geo_accession':'data_accession',
    'characteristics_ch1.4':'age',
    'characteristics_ch1.3':'gender',
    'Status':'desease', 
    'contact_city':'origin',
    'library_strategy':'METHOD', 
    'source_name_ch1':'TYPE',
    'platform_id':'PLATFORM_GEO_ID',
    'instrument_model':'PLATFORM_DESCRIPTION',
})
wimmers_GEO239787_df['study_accession']='GSE239787'
wimmers_GEO239787_df['origin']='Contact City: Palo Alto'
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age'].str.replace('age, months: ', '').astype(float)
wimmers_GEO239787_df['age']=wimmers_GEO239787_df['age']/12


wimmers_GEO239787_df_reduced = reorder_columns_by_metadata_and_gene_counts(wimmers_GEO239787_df)

print(wimmers_GEO239787_df_reduced.shape)
wimmers_GEO239787_df_reduced.tail()



(90, 60707)
(90, 60694)


,study_accession,subject_accession,data_accession,age,gender,desease,origin,METHOD,TYPE,PLATFORM_DESCRIPTION,...,ENSG00000273532,ENSG00000276351,ENSG00000275661,ENSG00000277856,ENSG00000275063,ENSG00000271254,ENSG00000275405,ENSG00000275987,ENSG00000277475,ENSG00000268674
85,GSE239787,CC0146,GSM7673286,1.900000,Sex: Female,acute-omicron,Contact City: Palo Alto,RNA-Seq,Whole blood,Illumina NovaSeq 6000,...,0,0,0,292,110,225,3,0,0,0
86,GSE239787,CC0261,GSM7673287,1.475000,Sex: Female,acute-omicron,Contact City: Palo Alto,RNA-Seq,Whole blood,Illumina NovaSeq 6000,...,0,0,0,179,314,178,2,0,0,0
87,GSE239787,CC0191,GSM7673288,0.541667,Sex: Male,acute,Contact City: Palo Alto,RNA-Seq,Whole blood,Illumina NovaSeq 6000,...,0,0,0,147,414,180,0,1,5,0
88,GSE239787,CC0022,GSM7673289,1.066667,Sex: Female,acute,Contact City: Palo Alto,RNA-Seq,Whole blood,Illumina NovaSeq 6000,...,0,0,0,247,256,86,1,0,0,0
89,GSE239787,CC0184,GSM7673290,0.525000,Sex: Female,acute,Contact City: Palo Alto,RNA-Seq,Whole blood,Illumina NovaSeq 6000,...,0,0,0,0,6,196,2,0,2,0


In [58]:
# 2020 Arunachalam, Wimmers
#NOTE migh had some issues in the prep df file, but the GEO actually did a good job 
#NOTE I can probably add 4 more subjects but want to clarify with flo first
aranachalam_2020_GSE152418_df = pd.read_csv("2020_Arunachalam_Wimmers/2020_aranachalam_GSE152418_combi.csv", index_col=0)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df = aranachalam_2020_GSE152418_df.rename(columns={
    'title':'subject_accession', 
    'geo_accession':'data_accession',
    'Age':'age',
    'characteristics_ch1.2':'gender',
    'characteristics_ch1.3':'desease', 
    'contact_city':'origin',
    'library_strategy':'METHOD', 
    'source_name_ch1':'TYPE',
    'instrument_model':'PLATFORM_DESCRIPTION',
    'platform_id':'PLATFORM_GEO_ID',
})
aranachalam_2020_GSE152418_df['study_accession'] = 'GSE152418'
aranachalam_2020_GSE152418_df = reorder_columns_by_metadata_and_gene_counts(aranachalam_2020_GSE152418_df)
print(aranachalam_2020_GSE152418_df.shape)
aranachalam_2020_GSE152418_df.head()

/tmp/ipykernel_126545/2573711352.py:4: DtypeWarning: Columns (60729,60730,60735) have mixed types. Specify dtype option on import or set low_memory=False.
  aranachalam_2020_GSE152418_df = pd.read_csv("2020_Arunachalam_Wimmers/2020_aranachalam_GSE152418_combi.csv", index_col=0)


(24, 60739)
(24, 60694)


,study_accession,subject_accession,data_accession,age,gender,desease,origin,METHOD,TYPE,PLATFORM_DESCRIPTION,...,ENSG00000273532,ENSG00000276351,ENSG00000275661,ENSG00000277856,ENSG00000275063,ENSG00000271254,ENSG00000275405,ENSG00000275987,ENSG00000277475,ENSG00000268674
1,GSE152418,S147_nCoV001EUHM-Draw-1,GSM4614986,75.0,gender: F,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,0,0,0,174,151,31,22,0,0,0
2,GSE152418,S149_nCoV002EUHM-Draw-2,GSM4614987,54.0,gender: M,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,0,0,0,283,1313,56,13,0,0,0
3,GSE152418,S150_nCoV003EUHM-Draw-1,GSM4614988,75.0,gender: F,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,0,0,0,1365,5028,42,19,0,0,0
4,GSE152418,S151_nCoV004EUHM-Draw-1,GSM4614989,59.0,gender: M,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,0,0,0,184,2240,42,5,0,1,0
5,GSE152418,S152_nCoV006EUHM-Draw-1,GSM4614990,59.0,gender: M,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,0,0,0,346,5159,66,4,0,0,0


In [98]:
# 2021 Thair COVID 
# NOTE 

thair_2021_GSE152641_df = pd.read_csv("2021_Thair_COVID/thair_combi_covid.csv", low_memory=False, index_col=0)
print(thair_2021_GSE152641_df.shape)
thair_2021_GSE152641_df = thair_2021_GSE152641_df.rename(columns={
      'title':'subject_accession', 
       'geo_accession':'data_accession',
       'characteristics_ch1.3':'age',
       'characteristics_ch1.4':'gender',
       'characteristics_ch1':'desease',
      'contact_city':'origin',
      'library_strategy':'METHOD',
      'characteristics_ch1.2':'TYPE',
      'instrument_model':'PLATFORM_DESCRIPTION',
      'platform_id':'PLATFORM_GEO_ID',
})
thair_2021_GSE152641_df['study_accession'] ='GSE152641'
thair_2021_GSE152641_ordered_df = reorder_columns_by_metadata_and_gene_counts(thair_2021_GSE152641_df)
print(thair_2021_GSE152641_ordered_df.shape)
thair_2021_GSE152641_ordered_df.head()

(89, 20502)
(89, 20256)


,study_accession,subject_accession,data_accession,age,gender,desease,origin,METHOD,TYPE,PLATFORM_DESCRIPTION,...,ENSG00000210151,ENSG00000286140,ENSG00000286920,ENSG00000284791,ENSG00000242288,ENSG00000267281,ENSG00000285480,ENSG00000285292,ENSG00000286264,ENSG00000287363
0,GSE152641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,113219467,113455421,113523636,113523638,113939925,114108587,114483833,114483834,114841035,114841037
1,GSE152641,IMX_sample00001,GSM4622627,age: 46,Sex: Male,disease: Healthy control,Burlingame,RNA-Seq,tissue: Whole blood,Illumina NovaSeq 6000,...,0,12,1004,0,0,5,0,0,0,0
2,GSE152641,IMX_sample00002,GSM4622628,age: 61,Sex: Male,disease: Healthy control,Burlingame,RNA-Seq,tissue: Whole blood,Illumina NovaSeq 6000,...,0,14,1063,0,0,9,1,0,0,0
3,GSE152641,IMX_sample00003,GSM4622629,age: 37,Sex: Male,disease: Healthy control,Burlingame,RNA-Seq,tissue: Whole blood,Illumina NovaSeq 6000,...,0,50,993,1,0,8,0,0,0,0
4,GSE152641,IMX_sample00004,GSM4622630,age: 48,Sex: Male,disease: Healthy control,Burlingame,RNA-Seq,tissue: Whole blood,Illumina NovaSeq 6000,...,0,26,904,0,0,0,2,0,0,0


### Concat together

This will contain unprocessed concatinated file with raw counts from rna seq. 
### TODO´s:
- preprocessing
- unify entries for everything
- NA and NAN in gene counts 
- douplicates (as there will also be longitudinal data) (check for "draw as well")


In [99]:
concatinated_df = pd.concat([aranachalam_2020_GSE152418_df, wimmers_GEO239787_df_reduced, thair_2021_GSE152641_df])
print(concatinated_df.shape)
concatinated_df.head()




(203, 61872)


,study_accession,subject_accession,data_accession,age,gender,desease,origin,METHOD,TYPE,PLATFORM_DESCRIPTION,...,contact_institute,contact_address,contact_state,contact_zip/postal_code,contact_country,data_row_count,library_selection,library_source,relation,supplementary_file_1
1,GSE152418,S147_nCoV001EUHM-Draw-1,GSM4614986,75.0,gender: F,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GSE152418,S149_nCoV002EUHM-Draw-2,GSM4614987,54.0,gender: M,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GSE152418,S150_nCoV003EUHM-Draw-1,GSM4614988,75.0,gender: F,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GSE152418,S151_nCoV004EUHM-Draw-1,GSM4614989,59.0,gender: M,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GSE152418,S152_nCoV006EUHM-Draw-1,GSM4614990,59.0,gender: M,disease state: COVID-19,Atlanta,RNA-Seq,PBMC,Illumina NovaSeq 6000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
